In [ ]:
import os
import sys
# print(sys.path)
import subprocess
import shutil
import pandas as pd
import numpy as np
import calendar
import traceback
import json
from pyutils import *
from pdutils import *
from pdpltutils import *
from gputils import *
from iputils import *
import xmltodict
import re
import pytesseract

from kmlmaker import KMLMaker as kmlm
from kmlproc import KMLCombiner as kmlc

In [ ]:
if sys.platform == "linux":
    DATADIR="/home/jblake1/Downloads/Network_Measurements"
else:
    DATADIR="P:\\My Drive\\CMU-LEL\\Mill19\\Coverage and Performance"
    EXPDIR=os.path.join(*[DATADIR,"2024-10-10-Mill19-Testing","LEL-UE1"])

DIRCHECKLIST=[DATADIR,EXPDIR]
for DIR in DIRCHECKLIST:
    print(f"{DIR} exists") if os.path.isdir(DIR) else print(f"{DIR} does not exist")

### KML File Processing

In [ ]:
EXPDIR=os.path.join(*[DATADIR,"2024-10-10-Mill19-Testing","LEL-UE1"])
def runUE(filename=None,filtervalue="Living_Edge_Lab"):
    kc = kmlc()
    # kc.setFiles(rxlevfiles)
    kc.findFiles(EXPDIR)
    kc.combine(ftype='merge')
    kc.inflate()
    kc.labelsOn()
    kc.filter(filterin = True, filtervalue=filtervalue)
    kc.writeKMLFile(filename=filename)
    return kc

In [ ]:

ffnlst = walkDir(EXPDIR)
# print(ffnlst)
pngfiles = [fn for fn in ffnlst if ".png" in fn]
kmlfiles =  [fn for fn in ffnlst if ".kml" in fn ]
txtfiles =  [fn for fn in ffnlst if ".txt" in fn ]

print(f"pngfiles={len(pngfiles)} kmlfiles={len(kmlfiles)} txtfiles={len(txtfiles)}")
rxlevfiles = [fn for fn in kmlfiles if "rxlev.kml" in fn]
print(rxlevfiles)

In [ ]:
filename = 'MadeOutput' + "_".join(EXPDIR.split("\\")[-2:]) + '.kml'
print(filename)
kc = runUE(filename=filename,filtervalue="314737")

### Termux Ping Logs

In [ ]:
pingtxtfiles = [fn for fn in txtfiles if "ping" in fn]
# print(pingtxtfiles)

In [ ]:
''' Combine Ping Files '''
tdft = pd.DataFrame(pingtxtfiles,columns=['FFN'])
kc = kmlc()
tdft['FILEDATA'] = tdft.FFN.map(kc.readFile)
columns = ["PINGTIMES"]
tdfx = pd.DataFrame(columns = columns)
for fdata in list(tdft.FILEDATA):
    sdata = [dline.strip("\n") for dline in fdata[1:]]
    tdfy = pd.DataFrame(sdata,columns = columns)
    tdfx = pd.concat([tdfx, tdfy],axis=0)

dumpdf(tdfx)


In [ ]:
print(tdfx.PINGTIMES.iloc[0])

In [ ]:
def parseTMXPingLine(row):
    text = row.PINGTIMES
    matchts = r"\[(.*?)\].*"
    matchpt = r".*time=(.*?) ms.*"
    return [float(re.sub(matchts, r'\1', text)),float(re.sub(matchpt, r'\1', text))]
tdfx[['LTIMESTAMP','PINGTIME']] = tdfx.apply(parseTMXPingLine,axis=1,result_type='expand')
# dumpdf(tdfx)
# Convert to datetime, assuming the timestamps are in seconds
tdfx['TIMESTAMP'] = pd.to_datetime(tdfx['LTIMESTAMP'], unit='s')
# Set the timezone (e.g., to 'US/Eastern')
tdfx['TIMESTAMP'] = tdfx['TIMESTAMP'].dt.tz_localize('UTC').dt.tz_convert('US/Eastern')
tdfx = tdfx[['TIMESTAMP','PINGTIME']]

In [ ]:
# tdfx = tdfx[tdfx.PINGTIME < 1000]

In [ ]:
dumpdf(tdfx)

In [ ]:
def histplot(dfin,title='Unknown Title',ax=None,filename='tmp.png', 
    figsize=(5,5), xlabel='',ylabel='',tabon=True, saveon=False,legend=False,
    bins=10, alpha=0.5, fontsize = 15, yticks = True,
    tabfontsize = 30, tabsizex = 1,tabsizey=2,**kwargs):
    font = {'size':fontsize}
    matplotlib.rc('font',**font)
    df = pd.DataFrame(dfin) # in case actually a series
    ''' Parameters '''
    if ax is None:
        ax = plt.figure(figsize=figsize).add_subplot(111)
    ''' Plot '''
    ax = df.plot.hist(bins=bins,alpha=alpha,title=title,figsize=figsize,ax=ax,legend=legend,**kwargs)
    ax.set_xlabel(xlabel)
    if not yticks: ax.set_yticklabels([])
    # print(tabon)
    if tabon:
        tabcolWidths = [0.2]
        tab = table(ax,np.round(df.describe(),2),loc='upper right',colWidths=tabcolWidths)
        tab.set_fontsize(tabfontsize)
        tab.scale(tabsizex,tabsizey)
    if saveon:
        print("Saving %s" % filename)
        savePlot(ax,filename)
    return ax

In [ ]:
title="Termux Ping Logs"
ax=None
histplot(tdfx[tdfx.PINGTIME < 100].PINGTIME,ax=ax,tabon=False, title=title,saveon=False,bins=100)

In [ ]:
print(kmlfiles)

In [ ]:
kc = kmlc()
kc.findFiles(EXPDIR)
kc.combine()
kc.filter(filterin = True, filtervalue="314737")
kc.filter(filterin = True, filtername="PHONE STATE", filtervalue="D")
# dumpdf(kc.rxlevdf)

In [ ]:
tdfy = pd.DataFrame(kc.kresult['kml']['Folder']['Placemark'])

In [ ]:
def getDataKeyValue(key,extralst):
    for edata in extralst['Data']:
        if edata['@name'] == key:
            return edata['value']
    return None

tdfy['TIMESTAMP'] = tdfy.ExtendedData.map(lambda xx: getDataKeyValue('TIME',xx))
tdfy = to_ts(tdfy,format = '%Y.%m.%d_%H.%M.%S',origtz = "US/Eastern", newtz="US/Eastern")

In [ ]:
dumpdf(tdfy)
dumpdf(tdfx)

In [ ]:
tdfz = pd.merge_asof( tdfy.copy().sort_values('TIMESTAMP'),tdfx.copy().sort_values('TIMESTAMP'), on='TIMESTAMP',direction='nearest').copy()

In [ ]:
tdfz.iloc[98:100].ExtendedData.map(print)

key="PINGTIME"
tdfz = tdfz.apply(kc.addDataKeyValue,axis=1, inkey=key)

tdfz.iloc[98:100].ExtendedData.map(print)

In [ ]:
dumpdf(tdfz)

In [ ]:
newdatalst = list(tdfz.ExtendedData.map(lambda xx: xx['Data']))
kc.replaceExtendedData(newdatalst)

In [ ]:
kc.inflate()
kc.labelsOn()
kc.filter(filterin = True, filtervalue="314737")

In [ ]:
kc.writeKMLFile(filename=filename)